In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn import model_selection, metrics
import transformers
import torch

In [3]:
import warnings 
warnings.filterwarnings('ignore')

In [4]:
class TextDataset:
    
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        enc = tokenizer(
            row["text"],
            max_length=10,
            truncation=True,
            padding="max_length"
        )
        
        return {
            "input_ids": torch.tensor(enc["input_ids"]),
            "attention_mask": torch.tensor(enc["attention_mask"]),
            # "label": torch.tensor(row["label"]),
        }

In [5]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv").rename(columns={"review": "text"})

id2label = {0: "negative", 1: "positive"}
label2id = {label: id_ for id_, label in id2label.items()}

df["label"] = df["sentiment"].map(label2id)

print(df.shape)
df.head()

(50000, 3)


,text,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [6]:
model_path = "/kaggle/input/imdb-review-classification-using-transformers/my-model/"

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_path)

In [8]:
ds = TextDataset(df)

In [9]:
[ds[0], ds[1]]

[{'input_ids': tensor([  101,  2028,  1997,  1996,  2060, 15814,  2038,  3855,  2008,   102]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'input_ids': tensor([ 101, 1037, 6919, 2210, 2537, 1012, 1026, 7987, 1013,  102]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}]

In [10]:
dl = torch.utils.data.DataLoader(
    ds,
    batch_size=2,
    shuffle=False,
    num_workers=2,
)

In [11]:
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-1

In [12]:
for idx, batch in enumerate(dl):
    print(batch)
    
    batch = {k: v.to('cuda') for k, v in batch.items()}
    
    with torch.no_grad():
        out = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
    
    if idx == 5:
        break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'input_ids': tensor([[  101,  2028,  1997,  1996,  2060, 15814,  2038,  3855,  2008,   102],
        [  101,  1037,  6919,  2210,  2537,  1012,  1026,  7987,  1013,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  1045,  2245,  2023,  2001,  1037,  6919,  2126,  2000,   102],
        [  101, 10468,  2045,  1005,  1055,  1037,  2155,  2073,  1037,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[ 101, 9004, 3334, 4717, 7416, 1005, 1055, 1000, 2293,  102],
        [ 101, 2763, 2026, 2035, 1011, 2051, 5440, 3185, 1010,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  1045,  2469,  2052,  2066,  2000,  2156,  1037, 15218,   102],
        [  101,  2023,  2265,  2001,  2019,  6429,  1010,  4840,  1004,   102]]), 'attention_mask'

In [13]:
batch

{'input_ids': tensor([[  101,  6316,  1996,  7344,  2003,  2028,  1997,  2216, 21864,   102],
         [  101,  1045,  2387,  2023,  3185,  2043,  1045,  2001,  2055,   102]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [14]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-1

In [15]:
out

SequenceClassifierOutput(loss=None, logits=tensor([[-1.2842,  1.3445],
        [-1.4889,  1.5748]], device='cuda:0'), hidden_states=None, attentions=None)

In [16]:
pipe = transformers.pipeline(
    task="text-classification",
    model=model_path,
    batch_size=4
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [17]:
test_case = ["I hated how good the movie was."]
result = pipe(test_case)

# Assuming the label comes as something like "LABEL_0" or "LABEL_1"
label_id = int(result[0]["label"].split("_")[1])  # Extract the ID part after "_"
result[0]["label"] = id2label[label_id]  # Map the ID to the label
result

[{'label': 'negative', 'score': 0.9627048969268799}]